# Transformer  (distilbert-base-uncased)

In [1]:
import os
import glob

import numpy as np
import pandas as pd

#import cudf
#import cupy as cp
import nvtabular as nvt

from google.cloud import bigquery

import pandas as pd
# import dask.dataframe as dd
# from dask.distributed import Client
# from dask.diagnostics import ProgressBar
# client = Client(n_workers=4, threads_per_worker=4, memory_limit="1GB")
# client

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../key/key.json"

In [2]:
 %reload_ext google.cloud.bigquery

In [ ]:
 %%bigquery df_combined
 
SELECT 
COUNT(*) article_count,
STRING_AGG(DISTINCT CAST (FORMAT("%02d", article.article_id) AS STRING)) article_id,
ROUND(AVG(trans.price), 6) Avg_price,
CONCAT(STRING_AGG(DISTINCT article.prod_name), " ,", 
STRING_AGG(DISTINCT article.graphical_appearance_name), " ,",
STRING_AGG(DISTINCT article.colour_group_name), " ,", 
STRING_AGG(DISTINCT CAST (article.perceived_colour_value_name AS STRING))," ,", 
STRING_AGG(DISTINCT CAST (article.garment_group_name AS STRING))," ,",
STRING_AGG(DISTINCT article.index_name)," ,",
STRING_AGG(DISTINCT CAST (article.section_name AS STRING))) description,
STRING_AGG(Distinct detail_desc) detail_desc,
CONCAT(STRING_AGG(DISTINCT article.prod_name), " ,", 
STRING_AGG(DISTINCT article.graphical_appearance_name), " ,",
STRING_AGG(DISTINCT article.colour_group_name), " ,", 
STRING_AGG(DISTINCT CAST (article.perceived_colour_value_name AS STRING))," ,", 
STRING_AGG(DISTINCT CAST (article.garment_group_name AS STRING))," ,",
STRING_AGG(DISTINCT article.index_name)," ,",
STRING_AGG(DISTINCT CAST (article.section_name AS STRING)), ",",
STRING_AGG(Distinct detail_desc)) comb_desc,
FROM`data-buckeye-347917.fashion_recommender.transaction` AS trans
LEFT JOIN `data-buckeye-347917.fashion_recommender.articles` AS article
ON (trans.article_id  = article.article_id) 
Left JOIN `data-buckeye-347917.fashion_recommender.customer` AS custom
ON (trans.customer_id = custom.customer_id)
WHERE sales_channel_id = 2
GROUP BY trans.article_id
ORDER BY article_count DESC
LIMIT 1000;

There are around 817 different prices at max. So the average was taken.
 in addition only online bought products were observed. 

In [ ]:
df = pd.DataFrame(df_combined)
df.to_parquet("../parquet/transform_example_comb_all.parquet")

In [5]:
df=pd.read_parquet("../parquet/transform_example_comb_10000.parquet")

In [6]:
df.head()

,article_count,article_id,Avg_price,description,detail_desc,comb_desc
0,42544,706016001,0.032499,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,..."
1,29292,706016002,0.032449,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L..."
2,21950,610776002,0.008091,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...",T-shirt in lightweight jersey with a rounded h...,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L..."
3,19881,706016003,0.032700,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me..."
4,19462,562245001,0.026159,"Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","5-pocket jeans in washed, superstretch denim w...","Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,..."


In [7]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   article_count  10000 non-null  Int64  
 1   article_id     10000 non-null  object 
 2   Avg_price      10000 non-null  float64
 3   description    10000 non-null  object 
 4   detail_desc    9975 non-null   object 
 5   comb_desc      9975 non-null   object 
dtypes: Int64(1), float64(1), object(4)
memory usage: 6.9 MB


In [8]:
df.describe(include="object").T

,count,unique,top,freq
article_id,10000,10000,706016001,1
description,10000,9569,"CHARLIE SKIRT ,Solid ,Black ,Dark ,Skirts ,Div...",8
detail_desc,9975,4739,High-waisted jeans in washed superstretch deni...,34
comb_desc,9975,9633,"CHARLIE SKIRT ,Solid ,Black ,Dark ,Skirts ,Div...",8


In [9]:
df["detail_desc"].isnull().sum()

25

In [10]:
df["detail_desc"].fillna("0", inplace=True)

In [11]:
df["detail_desc"].isnull().sum()

0

In [12]:
df["comb_desc"].isnull().sum()

25

In [13]:
df["comb_desc"].fillna("0", inplace=True)

In [14]:
df["comb_desc"].isnull().sum()

0

## Transfromer

In [15]:
import texthero as hero
from texthero import preprocessing
from sentence_transformers import SentenceTransformer, util
import nmslib 
import time
import datetime

/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/.capestone/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [16]:
distilbert = SentenceTransformer('distilbert-base-uncased')


Some weights of the model checkpoint at /Users/kaergell/.cache/torch/sentence_transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
embeddings = distilbert.encode(df['description'], convert_to_tensor=True)


In [18]:
embeddings2 =  distilbert.encode(df['detail_desc'], convert_to_tensor=True)

In [19]:
embeddings3=  distilbert.encode(df['comb_desc'], convert_to_tensor=True)

In [36]:
df['distilbert'] = np.array(embeddings).tolist()
df['distilbert_2'] = np.array(embeddings2).tolist()
df['distilbert_3'] = np.array(embeddings3).tolist()

In [21]:
df.head()

,article_count,article_id,Avg_price,description,detail_desc,comb_desc,distilbert,distilbert_2,distilbert_3
0,42544,706016001,0.032499,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...","[0.3121841251850128, -0.2658127248287201, 0.25...","[0.12130171805620193, -0.35272854566574097, 0....","[0.1301654577255249, -0.30701515078544617, 0.2..."
1,29292,706016002,0.032449,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...","[0.29146721959114075, -0.292407363653183, 0.25...","[0.12130171805620193, -0.35272854566574097, 0....","[0.12508580088615417, -0.3234410881996155, 0.2..."
2,21950,610776002,0.008091,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...",T-shirt in lightweight jersey with a rounded h...,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...","[0.04739042744040489, -0.31804966926574707, 0....","[0.03639470785856247, -0.8530430793762207, -0....","[-0.10051365941762924, -0.6184231042861938, 0...."
3,19881,706016003,0.032700,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...","[0.23938877880573273, -0.2345537692308426, 0.2...","[0.12130171805620193, -0.35272854566574097, 0....","[0.10291185230016708, -0.2877957224845886, 0.2..."
4,19462,562245001,0.026159,"Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","5-pocket jeans in washed, superstretch denim w...","Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","[0.31111642718315125, -0.3668121099472046, 0.3...","[0.033310580998659134, -0.40976640582084656, 0...","[0.11035124957561493, -0.4645553529262543, 0.3..."


In [22]:
distilbert_index = nmslib.init(method='hnsw', space='cosinesimil')
distilbert_index.addDataPointBatch(embeddings)
distilbert_index.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

In [23]:
print(distilbert_index)

<nmslib.FloatIndex method='hnsw' space='cosinesimil' at 0x7f84c7da1920>


In [24]:
distilbert_index2 = nmslib.init(method='hnsw', space='cosinesimil')
distilbert_index2.addDataPointBatch(embeddings2)
distilbert_index2.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
****************************************************

In [25]:
distilbert_index3 = nmslib.init(method='hnsw', space='cosinesimil')
distilbert_index3.addDataPointBatch(embeddings3)
distilbert_index3.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
****************************************************


In [38]:
 #Save a meta index and the data
distilbert_index.saveIndex('index.bin', save_data=True)
 
 #Re-intitialize the library, specify the space
 newIndex = nmslib.init(method='hnsw', space='cosinesimil_sparse')
 #Re-load the index and the data
 newIndex.loadIndex('sparse_index.bin', load_data=True)

ValueError: The space type cosinesimil_sparse is not compatible with the type DENSE_VECTOR, only dense vector spaces are allowed!

# Creating a Search Function

In [27]:
def search_cloth(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index.knnQuery(query, k=12)

    matches = [] 
   
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detail_desc' : df.detail_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [28]:
search_cloth(df," 706016001")

,article_id,article_count,price,description,detail_desc,distance
0,679854011,1670,0.014231,"Maja Cardigan TVP ,Melange ,Light Grey ,Dusty ...","Long-sleeved cardigan in a soft, loose knit wi...",0.420265
1,660599002,3904,0.030907,"Baraboom (1) ,Melange ,Grey ,Dusty Light ,Knit...","Long cardigan in a soft, fine knit with long s...",0.422428
2,678942058,667,0.016528,"Harrison (1) ,Melange ,Off White ,Dusty Light ...","Short-sleeved jumper in a soft, fine-knit visc...",0.424144
3,741985005,1049,0.024555,"Barry cardigan (1) ,Solid ,Dark Green ,Medium ...","Cardigan in a soft, fine-knit viscose blend wi...",0.424637
4,616711001,585,0.032592,"ED Baraboom cardigan ,Melange ,Grey ,Dusty Lig...","Long cardigan in a soft, fine knit with long s...",0.424852
5,660599013,2821,0.030499,"Baraboom (1) ,Melange ,Light Beige ,Dusty Ligh...","Long cardigan in a soft, fine knit with long s...",0.425317
6,679854002,662,0.014752,"Maja Cardigan TVP ,Melange ,Black ,Dark ,Knitw...","Long-sleeved cardigan in a soft, loose knit wi...",0.425947
7,660599007,635,0.032846,"Baraboom (1) ,Melange ,Blue ,Medium ,Knitwear ...","Long cardigan in a soft, fine knit with long s...",0.426362
8,741985006,945,0.024750,"Barry cardigan (1) ,Solid ,Light Pink ,Dusty L...","Cardigan in a soft, fine-knit viscose blend wi...",0.426429
9,579541075,1495,0.016396,"Calista (1) ,Melange ,Light Grey ,Dusty Light ...","Cardigan in a soft, fine-knit viscose blend wi...",0.427004


In [40]:
df.columns

Index(['article_count', 'article_id', 'Avg_price', 'description',
       'detail_desc', 'comb_desc', 'distilbert', 'distilbert_2',
       'distilbert_3', 'tsnedistilbert'],
      dtype='object')

In [39]:
df['tsnedistilbert'] = hero.tsne(df['distilbert'])

In [47]:
df['tsnedistilbert_detail'] = hero.tsne(df['distilbert_2'])

In [48]:
df['tsnedistilbert_comb'] = hero.tsne(df['distilbert_3'])

In [54]:
df.columns

Index(['article_count', 'article_id', 'Avg_price', 'description',
       'detail_desc', 'comb_desc', 'distilbert', 'distilbert_2',
       'distilbert_3', 'tsnedistilbert', 'tsnedistilbert_detail',
       'tsnedistilbert_comb'],
      dtype='object')

In [55]:
hero.scatterplot(df, col='tsnedistilbert',color="Avg_price",title="descripton")

In [56]:
hero.scatterplot(df, col='tsnedistilbert_detail',color="Avg_price",title="detail")

In [57]:
hero.scatterplot(df, col='tsnedistilbert_comb',color="Avg_price",title="comb" )

For detail descript

In [29]:
def search_cloth2(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index2.knnQuery(query, k=12)
    
    matches = []    
    
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detail_desc' : df.detail_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [30]:
search_cloth2(df, "706016001")

,article_id,article_count,price,description,detail_desc,distance
0,747737001,2656,0.009843,"B Leila Sarong ,Solid ,Black ,Dark ,Swimwear ,...",Sarong in an airy weave. Size 130x150 cm.,0.365894
1,812464001,3121,0.009716,"B Leila Sarong ,Solid ,Black ,Dark ,Swimwear ,...",Sarong in an airy weave. Size 130x150 cm.,0.365894
2,747737002,670,0.009819,"B Leila Sarong ,Solid ,Dark Green ,Dark ,Swimw...",Sarong in an airy weave. Size 130x150 cm.,0.365894
3,855857001,955,0.013114,"B SASSY SARONG. ,All over pattern ,Dark Grey ,...",Sarong in an airy weave. Size 130x150 cm.,0.365894
4,473507001,1281,0.020047,"Lindsay shopper ,Solid ,Black ,Dark ,Accessori...",Shopper in supple imitation leather with two h...,0.402839
5,631902006,936,0.049548,"Majken loafer ,Solid ,Black ,Dark ,Shoes ,Ladi...","Loafers with pointed toes. Fabric linings, imi...",0.403033
6,633152001,1044,0.031448,"Selena shopper (1) ,Solid ,Black ,Dark ,Access...",Shopper in grained imitation leather with imit...,0.407054
7,633152012,1393,0.032050,"Selena shopper (1) ,Solid ,Black ,Dark ,Access...",Shopper in grained imitation leather with imit...,0.407054
8,633152005,619,0.023948,"Selena shopper (1) ,Solid ,Light Pink ,Dusty L...",Shopper in grained imitation leather with imit...,0.407054
9,633152003,736,0.026107,"Selena shopper (1) ,Solid ,Light Grey ,Medium ...",Shopper in grained imitation leather with imit...,0.407054


In [31]:
def search_cloth3(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index3.knnQuery(query, k=12)
    
    matches = []    
    
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detail_desc' : df.detail_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [32]:
search_cloth3(df, "706016001")

,article_id,article_count,price,description,detail_desc,distance
0,633152003,736,0.026107,"Selena shopper (1) ,Solid ,Light Grey ,Medium ...",Shopper in grained imitation leather with imit...,0.414580
1,633152012,1393,0.032050,"Selena shopper (1) ,Solid ,Black ,Dark ,Access...",Shopper in grained imitation leather with imit...,0.421176
2,633152001,1044,0.031448,"Selena shopper (1) ,Solid ,Black ,Dark ,Access...",Shopper in grained imitation leather with imit...,0.421176
3,631902006,936,0.049548,"Majken loafer ,Solid ,Black ,Dark ,Shoes ,Ladi...","Loafers with pointed toes. Fabric linings, imi...",0.421996
4,633152005,619,0.023948,"Selena shopper (1) ,Solid ,Light Pink ,Dusty L...",Shopper in grained imitation leather with imit...,0.423355
5,695420001,984,0.050781,"Cardeo ,Solid ,Black ,Dark ,Shoes ,Ladies Acce...",Ankle boots in imitation leather with elastic ...,0.426568
6,684238003,908,0.024464,"Savannah shopper (1) ,Solid ,Black ,Dark ,Acce...",Spacious shopper in soft imitation leather wit...,0.429726
7,723824004,676,0.024051,"Monica sandalette ,Solid ,Beige ,Dusty Light ,...",Imitation suede sandals with covered heels and...,0.429756
8,723824002,625,0.024015,"Monica sandalette ,Solid ,Black ,Dark ,Shoes ,...",Imitation suede sandals with covered heels and...,0.431895
9,468977003,693,0.018658,"Selena Shopper ,Solid ,Light Grey ,Dusty Light...",Reversible shopper with a grained imitation le...,0.432187


In [33]:
df['tsnedistilbert'] = hero.tsne(df['distilbert'])

In [34]:
df.head()

,article_count,article_id,Avg_price,description,detail_desc,comb_desc,distilbert,distilbert_2,distilbert_3,tsnedistilbert
0,42544,706016001,0.032499,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...","[0.3121841251850128, -0.2658127248287201, 0.25...","[0.12130171805620193, -0.35272854566574097, 0....","[0.1301654577255249, -0.30701515078544617, 0.2...","[-40.3271598815918, 11.248001098632812]"
1,29292,706016002,0.032449,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...","[0.29146721959114075, -0.292407363653183, 0.25...","[0.12130171805620193, -0.35272854566574097, 0....","[0.12508580088615417, -0.3234410881996155, 0.2...","[-39.744930267333984, 11.290761947631836]"
2,21950,610776002,0.008091,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...",T-shirt in lightweight jersey with a rounded h...,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...","[0.04739042744040489, -0.31804966926574707, 0....","[0.03639470785856247, -0.8530430793762207, -0....","[-0.10051365941762924, -0.6184231042861938, 0....","[65.32915496826172, 15.001702308654785]"
3,19881,706016003,0.032700,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...","[0.23938877880573273, -0.2345537692308426, 0.2...","[0.12130171805620193, -0.35272854566574097, 0....","[0.10291185230016708, -0.2877957224845886, 0.2...","[-40.411460876464844, 10.344768524169922]"
4,19462,562245001,0.026159,"Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","5-pocket jeans in washed, superstretch denim w...","Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","[0.31111642718315125, -0.3668121099472046, 0.3...","[0.033310580998659134, -0.40976640582084656, 0...","[0.11035124957561493, -0.4645553529262543, 0.3...","[-11.377948760986328, -9.798619270324707]"


In [35]:
#create scatter plot of wines using the 
hero.scatterplot(df, col='tsnedistilbert'
                 , article_id='article_id'
                 , title="Cloth Explorer"
                 , hover_data = ['title','article_id','price','description'])

TypeError: scatterplot() got an unexpected keyword argument 'article_id'

# With all 

In [ ]:
df_all=pd.read_parquet("../parquet/transform_example_comb_all.parquet")

In [ ]:
df_all["comb_desc"].fillna("0", inplace=True)

In [ ]:
df_all["detail_desc"].fillna("0", inplace=True)

In [ ]:
all_embeddings = distilbert.encode(df_all['description'], convert_to_tensor=True)


In [ ]:
all_embeddings2 =  distilbert.encode(df_all['detail_desc'], convert_to_tensor=True)

In [ ]:
all_embeddings3=  distilbert.encode(df_all['comb_desc'], convert_to_tensor=True)

KeyboardInterrupt: 

In [ ]:
df_all['distilbert_3'] = np.array(all_embeddings3).tolist()

In [ ]:
all_distilbert_index = nmslib.init(method='hnsw', space='cosinesimil')
all_distilbert_index.addDataPointBatch(all_embeddings3)
all_distilbert_index.createIndex({'post': 2}, print_progress=True)

In [ ]:
def search_cloth(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances =all_ distilbert_index.knnQuery(query, k=12)

    matches = [] 
   
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detail_desc' : df.detail_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [ ]:
search_cloth(df_all," 706016001")